# Deaths and Hospitals

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
from matplotlib import dates
import matplotlib.pyplot as plt
import seaborn as sns
import sys; sys.path.append('../')
from src.data_loader.data_loader import load_google_mobility, load_deaths, load_interventions, load_counties, load_google_mobility_time_series
from src.utils.dates import get_today, lag_date, date2str, str2date, get_format
from src.utils.df_utils import get_date_columns
from src.pandas.align import align_lagged_dates

from scipy.stats import spearmanr

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [27]:
# Time series data
mobility, mobility_date = load_google_mobility()
deaths, deaths_date = load_deaths(join_county_codes=False)
interventions, interventions_date = load_interventions()

# Static data
counties, counties_date = load_counties()

# Processed mobility -> time series
mobility_ts, mobility_ts_date = load_google_mobility_time_series()

print(f'Mobility Last Updated {mobility_ts_date}')
print(f'Deaths Last Updated {deaths_date}')

Mobility Last Updated 04-26
Deaths Last Updated 04-26


In [4]:
death_dates = get_date_columns(mobility_ts, return_dtimes=False)

In [94]:
cum_deaths = pd.concat((deaths['FIPS'], deaths[death_dates].interpolate('linear', axis=1).sum(axis=1)), axis=1)
cum_deaths.columns = ['FIPS','cum_deaths']

In [95]:
cum_deaths = pd.merge(cum_deaths, counties[['FIPS','Total Hospitals (2019)']], on='FIPS')

In [96]:
cum_deaths[['cum_deaths','Total Hospitals (2019)']].corr('spearman')

,cum_deaths,Total Hospitals (2019)
cum_deaths,1.000000,0.567615
Total Hospitals (2019),0.567615,1.000000


In [98]:
confounders = [
    'POP_ESTIMATE_2018',
    'Density per square mile of land area - Population',
    'Rural-urban_Continuum Code_2013',
    'Total_age65plus',
]
cum_deaths = pd.merge(cum_deaths, counties[['FIPS'] + confounders], on='FIPS')

In [104]:
def normalize(row):
    deaths = row['cum_deaths']
    pop = row['POP_ESTIMATE_2018']
    density = row['Density per square mile of land area - Population']
    return(deaths / pop / density)

In [105]:
cum_deaths['normalized_deaths'] = cum_deaths.apply(lambda row: normalize(row), axis=1)

In [106]:
cum_deaths[['normalized_deaths', 'Total Hospitals (2019)']].corr('spearman')

,normalized_deaths,Total Hospitals (2019)
normalized_deaths,1.000000,0.452781
Total Hospitals (2019),0.452781,1.000000
